# Servicios cognitivos de Speech Recognition

## ¿Que son los servicios cognitivos?

Crear modelos de Computer Vision que tengan gran precisión y funcionen en una variedad de contextos importante no es una tarea sencilla. Requiren grandes cantidades de datos y experencia en la creación de modelos básados en Deep Learning. A medida que el procesamiento de imagenes se vuelve mas robusto, muchas organizaciones de tecnología han optado por empaquetar estos modelos de alta performance y ofrecerlos como servicios que las organizaciones pueden consumir directamente como un servicio web.

Azure Cognitive Services es uno de estos servicios, el cual cuenta con una API y librerias que se pueden utilizar en multiples lenguajes para ayudarnos a construir aplicaciones inteligentes sin necesidad de tener habilidades de ciencia de datos ni grandes volumenes de información.

### Creación de una cuenta en Azure

Hay varias alternativas para utilizar los servicios cognitivos de Azure, pero en nuestro caso crearemos una cuenta de pruebas que tendrá una duracción de 7 días para probar los servicios cognitivos. Esta metodología no requiere de una tarjeta de crédito. Si necesitaran disponer de una cuenta por mayor cantidad de tiempo, es posible crear una cuenta gratuita de Azure que dispone de acceso a los servicios en un Tier Free por 12 meses, sin embargo, este procedimiento requiere la validación del usuario por medio de una tarjeta de crédito.

Para crear una cuenta, utilicen el siguiente link: https://azure.microsoft.com/es-es/try/cognitive-services/

Luego debemos indicar que tipo de cuenta utilizaremos. en nuestro caso, especificaremos una cuente de tipo Trial que durará por 7 dias desde el momento de creación:

![Crear una cuenta de invitado](https://github.com/santiagxf/M72109/blob/master/docs/vision/_images/free-trial.png?raw=1)

Luego deberemos aceptar los terminos y condiciones para utilizar el servicio y finalmente iniciar sesión utilizando alguna de las opciones que están disponibles. Si no disponen de una cuenta Microsoft, pueden crearse una desde https://signup.microsoft.com y eligiendo la opción Crear una Cuenta Microsoft.

### Obtener las claves de acceso a los servicios

Una vez que activamos nuestra cuenta trial, obtendremos 2 puntos de información importante:
 - Puntos de conexión: Es la URL que deberemos de indicar para acceder al servicio
 - Claves: Estas claves son el mecanismo de autenticación que utilizaremos.

Necesitaremos ambas piezas de información para continuar. Si no pueden encontrar sus claves, pueden acceder a https://azure.microsoft.com/es-es/try/cognitive-services/my-apis y debería llevarlos a una pagina similar a la siguiente donde están especificados ambos valores:

![Obtener clases de acceso](https://github.com/santiagxf/M72109/blob/master/docs/audio/_images/speech-key.png?raw=1)

## Creación de un cliente para la utilización de la API de speech

### Instalación del SDK

Instalamos las librerias del SDK junto con algunas otras que necesitaremos

In [ ]:
!pip install azure-cognitiveservices-speech librosa

Importamos las clases con las que vamos a trabajar

In [ ]:
import os
import librosa
import azure.cognitiveservices.speech as speechsdk

Especificación de las credenciales para operar el servicio. Aquí deberemos utilizar las credenciales que generaron desde la página del portal de Microsoft Azure

In [ ]:
subscription_key = ""
endpoint = "https://westus.api.cognitive.microsoft.com"
region='westus'

Indiquemos el directorio en donde están nuestros archivos de ejemplo

In [ ]:
rootdir='./Samples'

Descarguemos un archivo de ejemplo:

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Samples/Back_To_The_Future_3648_3658_6.wav --directory-prefix ./Samples/

Especificamos un archivo puntual para analizar

In [ ]:
wave_file_name='Back_To_The_Future_3648_3658_6.wav'
wave_file=os.path.join(rootdir, wave_file_name)

Reproduzcamos el archivo:

In [ ]:
sample_rate = 44100
wave = librosa.load(wave_file, sr=sample_rate)

In [ ]:
from IPython.display import Audio
import numpy as np

Audio(wave[0].astype(np.float32), rate=sample_rate)

Creamos una configuración para ejecutar el reconocimiento de audio. Esta API tiene varios modos de operar y puede trabajar tanto con archivos de audio e incluso hacer

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
audio_config = speechsdk.AudioConfig(filename=wave_file)

Cofiguramos como el servicio debe manejar palabras que son inapropiadas para el lenguaje. En este caso los enmascaremos (asteriscos)

In [ ]:
speech_config.set_profanity(speechsdk.ProfanityOption.Masked)

In [ ]:
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

In [ ]:
result = speech_recognizer.recognize_once()

In [ ]:
if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Resultado del reconocimiento: {}".format(result.text))
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No se pudo reconocer el texto: {}".format(result.no_match_details))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Reconocimiento cancelado por la siguiente razón: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Errores: {}".format(cancellation_details.error_details))